In [ ]:


# Install
!pip install scikit-surprise --no-deps -q
!pip install scipy numpy scikit-learn -q

import numpy as np
import pandas as pd
import os, glob, pickle, time
from scipy import sparse
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
print("✅ All imports successful!")


print(" FIXED Training (Anti-Overfitting)")


# Clone data
if not os.path.exists('data'):
    !git clone https://github.com/SimpliSolve/ai-innovation-talent-hunt-dataset.git data

ML = 'data/ml-latest-small'
UR = 'data/user_reviews'
DC = 'data/daily_csvs'

# Load MovieLens
print("\n Loading MovieLens...")
ratings_df = pd.read_csv(f"{ML}/ratings.csv")
movies_df = pd.read_csv(f"{ML}/movies.csv")
links_df = pd.read_csv(f"{ML}/links.csv")
print(f"  {len(ratings_df):,} ratings")

# Load metadata (quick)
print("\n Loading metadata...")
movie_metadata = {}
for f in tqdm(glob.glob(f"{DC}/*.csv"), desc="Loading"):
    try:
        df = pd.read_csv(f, low_memory=False)
        for _,r in df.iterrows():
            l = r.get('movie_link','')
            if pd.notna(l) and l and l not in movie_metadata:
                movie_metadata[l] = {'name':r.get('movie_name',''), 'genre':r.get('genre','[]')}
    except: pass
print(f" {len(movie_metadata):,} movies")

# IMDB profiles
print("\n Building IMDB profiles...")
imdb_user_profiles = defaultdict(lambda: {'movies_reviewed':[], 'ratings':[], 'avg_rating':0, 'review_count':0})
total = 0
folders = sorted(os.listdir(UR)) if os.path.exists(UR) else []
for mf in tqdm(folders, desc="Processing"):
    fp = os.path.join(UR, mf)
    if not os.path.isdir(fp): continue
    ml = f"https://www.imdb.com/title/{mf}/"
    for cf in glob.glob(os.path.join(fp,'*.csv')):
        try:
            df = pd.read_csv(cf, encoding='utf-8', on_bad_lines='skip')
            for _,r in df.iterrows():
                uid = str(r.get('User_id','')).strip()
                rs = str(r.get('User_rating','')).strip()
                if not uid or not rs or uid=='nan': continue
                try: rt = float(rs.split('/')[0]) if '/' in rs else float(rs)
                except: continue
                imdb_user_profiles[uid]['movies_reviewed'].append(ml)
                imdb_user_profiles[uid]['ratings'].append(rt)
                imdb_user_profiles[uid]['review_count'] += 1
                total += 1
        except: pass
for u,p in imdb_user_profiles.items():
    if p['ratings']: p['avg_rating'] = np.mean(p['ratings'])
print(f" {total:,} reviews from {len(imdb_user_profiles):,} users")

# Features (same as before)
print("\n Creating features...")
all_genres = set()
for g in movies_df['genres']:
    if g != '(no genres listed)': all_genres.update(g.split('|'))
genres_list = sorted(list(all_genres))
n_genres = len(genres_list)
genre_to_idx = {g:i for i,g in enumerate(genres_list)}

users = sorted(ratings_df['userId'].unique())
items = sorted(ratings_df['movieId'].unique())
n_users, n_items = len(users), len(items)
user_to_idx = {u:i for i,u in enumerate(users)}
idx_to_user = {i:u for u,i in user_to_idx.items()}
item_to_idx = {m:i for i,m in enumerate(items)}
idx_to_item = {i:m for m,i in item_to_idx.items()}

imdb_to_ml, ml_to_imdb = {}, {}
for _,r in links_df.iterrows():
    mid, iid = int(r['movieId']), str(int(r['imdbId'])).zfill(7)
    l = f"https://www.imdb.com/title/tt{iid}/"
    imdb_to_ml[l] = mid; imdb_to_ml[l.rstrip('/')] = mid; imdb_to_ml[f"tt{iid}"] = mid
    ml_to_imdb[mid] = l

movie_genres_dict = {}
for _,r in movies_df.iterrows():
    gv = np.zeros(n_genres)
    if r['genres'] != '(no genres listed)':
        for g in r['genres'].split('|'):
            if g in genre_to_idx: gv[genre_to_idx[g]] = 1.0
    movie_genres_dict[r['movieId']] = gv

ir,ic,id_ = [],[],[]
for _,r in movies_df.iterrows():
    if r['movieId'] not in item_to_idx: continue
    idx = item_to_idx[r['movieId']]
    if r['genres'] != '(no genres listed)':
        for g in r['genres'].split('|'):
            if g in genre_to_idx: ir.append(idx); ic.append(genre_to_idx[g]); id_.append(1.0)
item_features = sparse.csr_matrix((id_,(ir,ic)), shape=(n_items,n_genres))

ugp = np.zeros((n_users,n_genres)); ugc = np.zeros((n_users,n_genres))
for _,r in ratings_df.iterrows():
    if r['userId'] in user_to_idx and r['movieId'] in movie_genres_dict:
        ui = user_to_idx[r['userId']]
        ugp[ui] += movie_genres_dict[r['movieId']] * (r['rating']/5.0)
        ugc[ui] += movie_genres_dict[r['movieId']]
with np.errstate(divide='ignore',invalid='ignore'):
    user_features_dense = np.where(ugc>0, ugp/ugc, 0)

rc = ratings_df.groupby('movieId').size()
ar = ratings_df.groupby('movieId')['rating'].mean()
mc = rc.max()
movie_popularity = {}
for mid in item_to_idx.keys():
    c,a = rc.get(mid,0), ar.get(mid,3.0)
    movie_popularity[mid] = {'count':int(c), 'avg_rating':float(a), 'popularity_score':float((c/mc)*(a/5.0)) if mc>0 else 0}
top_popular_movies = sorted(movie_popularity.keys(), key=lambda x: movie_popularity[x]['popularity_score'], reverse=True)[:100]
print(f" {n_users} users, {n_items} items, {n_genres} genres")

# 🔧 FIXED: Anti-Overfitting Training
print("\n🚀 Training SVD with ANTI-OVERFITTING settings...")
reader = Reader(rating_scale=(0.5, 5.0))
sdata = Dataset.load_from_df(ratings_df[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(sdata, test_size=0.2, random_state=42)

print("\n FIXED Hyperparameters (Anti-Overfitting):")
print("   OLD: n_factors=150, reg_all=0.02, lr_all=0.005")
print("   NEW: n_factors=100, reg_all=0.2, lr_all=0.003")
print("   NEW: Early stopping based on validation gap")

# Track with early stopping
eps = [5,10,15,20,25,30]
tr_h, te_h = [], []
best_gap = float('inf')
best_epoch = 0
best_model = None

print("\n Training with early stopping:")
for e in eps:
    # FIXED: Lower complexity, higher regularization
    m = SVD(n_factors=100, n_epochs=e, lr_all=0.003, reg_all=0.2, random_state=42)
    m.fit(trainset)
    tr = accuracy.rmse(m.test(trainset.build_testset()), verbose=False)
    te = accuracy.rmse(m.test(testset), verbose=False)
    gap = te - tr
    tr_h.append(tr); te_h.append(te)

    if gap < best_gap:
        best_gap = gap
        best_epoch = e
        best_model = m

    status = "✅" if gap < 0.15 else ("⚠️" if gap < 0.3 else "❌")
    print(f"  Epoch {e:2d}: Train={tr:.4f}, Test={te:.4f}, Gap={gap:.4f} {status}")

    # Early stop if gap is good
    if gap < 0.1:
        print(f"   Early stopping at epoch {e} (gap < 0.1)")
        break

print(f"\n Best epoch: {best_epoch} (gap: {best_gap:.4f})")

# Use best epoch for final model
bp = {'n_factors':100, 'n_epochs':best_epoch, 'lr_all':0.003, 'reg_all':0.2}
final_model = SVD(**bp, random_state=42)
final_model.fit(sdata.build_full_trainset())

# Evaluate
eval_model = SVD(**bp, random_state=42)
eval_model.fit(trainset)
tp = eval_model.test(testset)
rmse = accuracy.rmse(tp, verbose=False)
mae = accuracy.mae(tp, verbose=False)

# Recall
print("\n Recall@K...")
def recall_k(preds, k=5, th=3.5):
    ud = defaultdict(list)
    for p in preds: ud[p.uid].append((p.est, p.r_ui))
    recs = []
    for _,rats in ud.items():
        rats.sort(key=lambda x:x[0], reverse=True)
        nr = sum(t>=th for _,t in rats)
        nrr = sum((t>=th)and(e>=th) for e,t in rats[:k])
        if nr>0: recs.append(nrr/min(nr,k))
    return np.mean(recs) if recs else 0

rr = {}
for k in [1,3,5,10]:
    rr[k] = recall_k(tp, k=k)
    print(f"  Recall@{k}: {rr[k]:.4f}")

# Plots
print("\n Creating plots...")
fig = plt.figure(figsize=(14,10))

ax1 = fig.add_subplot(2,2,1)
ax1.plot(eps[:len(tr_h)], tr_h, 'b-o', label='Train', lw=2)
ax1.plot(eps[:len(te_h)], te_h, 'r-s', label='Test', lw=2)
ax1.fill_between(eps[:len(tr_h)], tr_h, te_h, alpha=0.3, color='yellow')
if best_epoch > 0:
    ax1.axvline(best_epoch, color='green', linestyle='--', linewidth=2, label=f'Best Epoch {best_epoch}')
ax1.set_xlabel('Epochs'); ax1.set_ylabel('RMSE')
ax1.set_title(' Overfitting Detection (FIXED)', fontweight='bold')
ax1.legend(); ax1.grid(True, alpha=0.3)
gap = te_h[-1]-tr_h[-1]
st = "GOOD" if gap<0.1 else ("⚠️MODERATE" if gap<0.2 else "❌OVERFIT")
ax1.annotate(f'{st}\nGap:{gap:.4f}', xy=(0.7,0.8), xycoords='axes fraction', fontweight='bold')

ax2 = fig.add_subplot(2,2,2)
ax2.bar(range(len(rr)), list(rr.values()), color='#2ecc71')
ax2.set_xticks(range(len(rr))); ax2.set_xticklabels([f'@{k}' for k in rr.keys()])
ax2.set_title(' Recall@K', fontweight='bold'); ax2.set_ylim(0,1)

ax3 = fig.add_subplot(2,2,3)
from sklearn.metrics import confusion_matrix
act = [p.r_ui for p in tp]; pred = [p.est for p in tp]
bins = [0.5,1.5,2.5,3.5,4.5,5.5]
cm = confusion_matrix(np.digitize(act,bins), np.digitize(pred,bins), labels=[1,2,3,4,5])
cmn = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
sns.heatmap(cmn, annot=True, fmt='.2f', cmap='Blues', ax=ax3, xticklabels=[1,2,3,4,5], yticklabels=[1,2,3,4,5])
ax3.set_xlabel('Predicted'); ax3.set_ylabel('Actual')
ax3.set_title(' Confusion Matrix', fontweight='bold')

ax4 = fig.add_subplot(2,2,4)
ax4.hist([p.est-p.r_ui for p in tp], bins=50, color='#9b59b6', alpha=0.7)
ax4.axvline(0, color='red', ls='--', lw=2)
ax4.set_xlabel('Error'); ax4.set_title(' Error Distribution', fontweight='bold')

plt.tight_layout()
plt.savefig('training_metrics_fixed.png', dpi=150)
plt.show()

# Results

print(f"  RMSE: {rmse:.4f} | MAE: {mae:.4f} | Recall@5: {rr[5]:.4f}")
print(f"  Status: {st} (Gap: {gap:.4f})")
print(f"  Best Epoch: {best_epoch}")


# Save
print("\n Saving model...")
os.makedirs('Resources', exist_ok=True)
idict = {k:dict(v) for k,v in imdb_user_profiles.items()}
md = {
    'svd_model':final_model, 'model_type':'svd_hybrid_fixed', 'hyperparams':bp,
    'n_users':n_users, 'n_items':n_items, 'n_genres':n_genres,
    'user_to_idx':user_to_idx, 'idx_to_user':idx_to_user,
    'item_to_idx':item_to_idx, 'idx_to_item':idx_to_item,
    'imdb_to_ml':imdb_to_ml, 'ml_to_imdb':ml_to_imdb,
    'genres':genres_list, 'genre_to_idx':genre_to_idx,
    'item_genre_matrix':item_features.toarray(),
    'user_genre_prefs':user_features_dense,
    'movie_genres_dict':movie_genres_dict,
    'movie_popularity':movie_popularity,
    'top_popular_movies':top_popular_movies,
    'imdb_user_profiles':idict,
    'movie_metadata':movie_metadata,
    'movie_names':dict(zip(movies_df['movieId'], movies_df['title'])),
    'performance':{'rmse':rmse,'mae':mae,'recall@5':rr[5],'gap':gap}
}
with open('Resources/hybrid_model.pkl','wb') as f: pickle.dump(md,f)
print(f" Saved: {os.path.getsize('Resources/hybrid_model.pkl')/(1024*1024):.2f} MB")

# Download
from google.colab import files
files.download('Resources/hybrid_model.pkl')
files.download('training_metrics_fixed.png')
